# Parlament Speaker Recogniction Model
This notebook should predict what speaker said certain things based on the Austrian parlament speaches. 

This small project is based on the Dataset assembled by Mario Zechner (https://marioslab.io/)

In [1]:
# DATA FROM SPEAKER
import pandas as pd

# Either use direct download or load locally https://woswormeileistung.marioslab.io/data/persons.json
#speaker = pd.read_json("https://woswormeileistung.marioslab.io/data/persons.json")
speaker = pd.read_json("persons.json")
print(speaker.shape)
speaker.head()

(681, 6)


,id,name,parties,periods,url,imageUrl
0,3612,Sonja Ablinger,[SPÖ],"[XX, XXIII, XXIV]",https://parlament.gv.at/person/3612,https://parlament.gv.at/dokument/bild/34898/34...
1,14854,Dipl.-Ing. Elke Achleitner,"[BZÖ, FPÖ]",[XXII],https://parlament.gv.at/person/14854,https://parlament.gv.at/dokument/bild/21013/21...
2,83119,Mag. Nikolaus Alm,"[LIF, NEOS]",[XXV],https://parlament.gv.at/person/83119,https://parlament.gv.at/dokument/bild/43808/43...
3,51879,"Mag. Hannes Amesbauer, BA",[FPÖ],"[XXVI, XXVII]",https://parlament.gv.at/person/51879,https://parlament.gv.at/dokument/bild/200696/2...
4,2819,"Werner Amon, MBA",[ÖVP],"[XIX, XX, XXI, XXII, XXIII, XXIV, XXV, XXVI]",https://parlament.gv.at/person/2819,https://parlament.gv.at/dokument/bild/90356/90...


As one can see there are about 681 Speakers from the austrian parlament, starting from 2002

In [2]:
# DATA FROM SESSIONS 
# BE AWARA: sessions data has 440 mb, so it is only imported locally 
# https://woswormeileistung.marioslab.io/data/sessions.json
# "\\wsl.localhost\Ubuntu-22.04\home\georg\git-projects\data\sessions.json"
sessions = pd.read_json("/home/georg/git-projects/data/sessions.json")

print(sessions.shape)
sessions.head()

(1001, 7)


,url,period,sessionNumber,sessionLabel,date,protocolUrls,sections
0,https://parlament.gv.at/gegenstand/XXVII/NRSIT...,XXVII,251,251. Sitzung (251/NRSITZ),2024-01-31,[],[]
1,https://parlament.gv.at/gegenstand/XXVII/NRSIT...,XXVII,250,250. Sitzung (250/NRSITZ),2024-01-31,[],[]
2,https://parlament.gv.at/gegenstand/XXVII/NRSIT...,XXVII,249,249. Sitzung (249/NRSITZ),2024-01-31,[],[]
3,https://parlament.gv.at/gegenstand/XXVII/NRSIT...,XXVII,248,248. Sitzung (248/NRSITZ),2023-12-15,[],[]
4,https://parlament.gv.at/gegenstand/XXVII/NRSIT...,XXVII,247,247. Sitzung (247/NRSITZ),2023-12-15,[],[]


In [3]:
sessions.sections.head()

0    []
1    []
2    []
3    []
4    []
Name: sections, dtype: object

In [4]:
sessions.sections.drop_duplicates()

0                                                      []
35      [{'speaker': '88386', 'text': 'Ich darf die 21...
36      [{'speaker': '88386', 'text': 'Meine sehr geeh...
39      [{'speaker': '88386', 'text': 'Ich eröffne die...
40      [{'speaker': '88386', 'text': 'Meine sehr geeh...
                              ...                        
996     [{'speaker': '799', 'text': 'Ich eröffne die 5...
997     [{'speaker': '799', 'text': 'Die Sitzung ist e...
998     [{'speaker': '799', 'text': 'Meine Damen und H...
999     [{'speaker': '799', 'text': 'Die Sitzung ist e...
1000    [{'speaker': '334', 'text': 'Meine Damen und H...
Name: sections, Length: 955, dtype: object

In [5]:
sessions.sections.describe()

count     1001
unique     955
top         []
freq        47
Name: sections, dtype: object

In [6]:
session_na = [1 if sessions["sections"][i] == [] else 0 for i in range(len(sessions))]
print("Absolute number of na: "+ str(sum(session_na)) + " || relative number of na: "+ str(sum(session_na)/len(sessions)))

Absolute number of na: 47 || relative number of na: 0.04695304695304695


In [7]:
# the entries in "speaker" arer list of dicts; therefore loop over the list to extract the dicts
text_dict = [j  for sections in sessions["sections"] for j in sections] 

# make a pandas df from the extracted dict
text = pd.DataFrame.from_dict(text_dict, orient="columns")
text["input"] = "SPEAKER: "+text["speaker"]+"; TEXT: "+text["text"]
print(text.shape) # 173498 
text.head()

(173498, 5)


,speaker,text,callouts,links,input
0,88386,Ich darf die 216. Sitzung des Nationalrates um...,"[{'caller': '2343', 'text': 'Ich bin noch nich...",[],SPEAKER: 88386; TEXT: Ich darf die 216. Sitzun...
1,88386,Hinsichtlich der eingelangten Verhandlungsgege...,[],"[{'label': '14141/AB', 'url': 'https://parlame...",SPEAKER: 88386; TEXT: Hinsichtlich der eingela...
2,88386,"Außerdem weise ich\n\ndie Anträge 3425/A, 3426...",[],"[{'label': '3425/A', 'url': 'https://parlament...",SPEAKER: 88386; TEXT: Außerdem weise ich\n\ndi...
3,88386,"Weiters darf ich mitteilen, dass folgende Fris...",[],"[{'label': '1774 der Beilagen', 'url': 'https:...",SPEAKER: 88386; TEXT: Weiters darf ich mitteil...
4,88386,Wir kommen nunmehr zur Abstimmung über den Ant...,[],"[{'label': '1774 der Beilagen', 'url': 'https:...",SPEAKER: 88386; TEXT: Wir kommen nunmehr zur A...


In [8]:
#for i in range(10): 
#    print(text["callouts"][i] == [])
tryout = pd.DataFrame([text["callouts"][i] == [] for i in range(10)])
tryout.describe()

,0
count,10
unique,2
top,True
freq,9


# Try the speaker classification with fast.ai library

In [9]:
# get all the speakers who have the most speaches
speaker_group = text[900:1000].drop(["callouts","links","input"],axis=1).groupby("speaker").count().sort_values(by="text",ascending=False)

# in the full data set if you take the speakers from 4th to 9th place you get roughly equally distribubted number of speaches
speaker_subset = speaker_group[3:8] 

speaker_subset

,text
speaker,
5682,3
5626,2
14835,2
83114,2
83129,2


In [10]:
train_df = text[text["speaker"].isin(speaker_subset.index)]
train_df.shape

(939, 5)

In [11]:
from fastai.text.data import TextDataLoaders
dl = TextDataLoaders.from_df(train_df, valid_pct=0.2, seed=None,text_col="text", label_col="speaker")
dl.show_batch(max_n=2)

,text,category
0,"xxbos xxmaj herr xxmaj präsident ( auf das xxmaj xxunk am xxmaj rednerpult xxunk ) , die xxmaj uhr rennt schon seit 10 xxmaj sekunden – auch schön ! xxmaj sehr geehrte xxmaj damen und xxmaj herren ! xxmaj ich denke mir , dass es sehr wichtig ist , dass die öffentliche xxmaj hand mit gutem xxmaj beispiel vorangeht , wenn es um nachhaltige xxmaj beschaffung geht . xxmaj sehr oft ist es auch die öffentliche xxmaj hand , die neue xxmaj benchmarks setzt , die dann überhaupt einmal ein xxmaj produkt , das ökologisch besser ist als der xxmaj durchschnitt , für xxmaj xxunk und xxunk xxunk und auch xxunk macht , weil vorher der xxmaj markt dafür zu klein gewesen wäre . \n\n xxmaj abgesehen davon denke ich mir , dass es wichtig ist , dass es mit grüner xxmaj beschaffung alleine nicht getan ist .",14835
1,"xxbos xxmaj herr xxmaj präsident ! xxmaj sehr geehrter xxmaj herr xxmaj bundesminister ! xxmaj ich glaube xxmaj ihnen – auch weil ich xxmaj sie schon länger kenne – , dass xxmaj sie kein xxmaj freund von xxmaj markteingriffen sind . – xxmaj das ist das , was xxmaj sie sagen . xxmaj das , was xxmaj sie tun , ist , dass xxmaj sie seit einiger xxmaj zeit – seit einigen xxmaj jahren – permanent xxmaj xxunk vornehmen . xxmaj das xxmaj xxunk , das wir jetzt haben , hat mit einer freien , sozialen xxmaj marktwirtschaft sehr , sehr wenig zu tun . \n\n xxmaj das hat mit dem xxmaj coronaregime begonnen – das war ein xxmaj bündel an xxmaj markteingriffen mit xxmaj lockdowns und so weiter und so fort . xxmaj auch die sogenannte xxmaj klimapolitik ist von ganz wesentlichen xxunk",83129


In [12]:
from fastai.metrics import accuracy,Perplexity
from fastai.text.learner import text_classifier_learner
from fastai.text.models import AWD_LSTM
learn = text_classifier_learner(dl, AWD_LSTM, metrics=[accuracy, Perplexity()])

In [ ]:
from fastai.text.all import *
learn.fit_one_cycle(1, 2e-2)

epoch,train_loss,valid_loss,accuracy,perplexity,time


In [ ]:
learn.predict("Ich bin Wolfgang Sobotka und ich versuche jetzt, ein bisschen ruhig zu bleiben, obwohl mir das zugegebenermaßen schwer fällt.")

In [ ]:
#speaker["id"] == 18140
speaker.loc[speaker["id"]==18140]

In [ ]:
len(train_df["speaker"].unique())

In [ ]:
# merge ids with speaker names
speaker_merge = pd.merge(speaker_subset, text, how='inner', on='speaker').groupby("speaker").count()
speaker_merge